In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os

In [ ]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip -qo hymenoptera_data.zip

--2020-07-27 07:17:27--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.224.153.12, 13.224.153.31, 13.224.153.85, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.224.153.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: ‘hymenoptera_data.zip.2’

hymenoptera_data.zi 100%[===================>]  45.10M   158MB/s    in 0.3s    

2020-07-27 07:17:27 (158 MB/s) - ‘hymenoptera_data.zip.2’ saved [47286322/47286322]



In [ ]:
data_transforms = transforms.Compose([
                                      transforms.RandomResizedCrop(224),
                                      transforms.ToTensor()])

data_dir = 'hymenoptera_data//train'
image_datasets = datasets.ImageFolder(os.path.join(data_dir), transform=data_transforms)
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=4, num_workers=4)
dat, lab = next(iter(dataloaders))

In [ ]:
model_ft = models.resnet18(pretrained=True)

In [ ]:
feature_out = {}
layers_name = list(model_ft._modules.keys())
layers = list(model_ft._modules.values())

def hook_fn_forward(module, input, output):
    layer_name = layers_name[np.argwhere([module == m for m in layers])[0, 0]]
    total_feat_out[layer_name] = output
    
modules = model_ft.named_children()
for name, module in modules:
    module.register_forward_hook(hook_fn_forward)

model_ft.eval() 
with torch.no_grad():
    pred = model_ft(dat)

In [ ]:
total_feat_out.keys()

dict_keys(['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3', 'layer4', 'avgpool', 'fc'])